<a href="https://colab.research.google.com/github/lucabem/CompeticionMineria/blob/main/Presentacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Cargar paquetes

In [2]:

!pip install --upgrade tensorflow_hub
!pip install --upgrade tensorflow_addons

import os
import tensorflow as tf
import tensorflow_hub as hub
from keras.preprocessing import image
import numpy as np
import pandas as pd
from google.colab import files
import tensorflow_addons as tfa
import pathlib



Requirement already up-to-date: tensorflow_hub in /usr/local/lib/python3.7/dist-packages (0.11.0)
Requirement already up-to-date: tensorflow_addons in /usr/local/lib/python3.7/dist-packages (0.12.1)


# 2. Cargar modelo

Se carga manualmente y se descomprime.

In [3]:
!unzip "model.zip"

Archive:  model.zip
   creating: model_2048_512_1024/
   creating: model_2048_512_1024/variables/
  inflating: model_2048_512_1024/variables/variables.index  
  inflating: model_2048_512_1024/variables/variables.data-00000-of-00001  
   creating: model_2048_512_1024/assets/
  inflating: model_2048_512_1024/saved_model.pb  


In [8]:
ls model_2048_512_1024/

assets/  saved_model.pb  variables/


In [9]:
path_to_model = "/content/model_2048_512_1024"
model = tf.keras.models.load_model(path_to_model, compile=True, custom_objects={'f1':tfa.metrics.F1Score(num_classes=25, average='weighted')})


In [10]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2048)              4196352   
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_3 (Dense)              (None, 25)                2

## 3. Cargar el Dataset

Se carga manualmente y se descomprime.

In [11]:
!unzip "test.zip"

Archive:  test.zip
   creating: test/
  inflating: test/Forest_CDN_17.jpg  
  inflating: test/Forest_CDN_18.jpg  
  inflating: test/Forest_CDN_19.jpg  
  inflating: test/Forest_CDN_2.jpg   
  inflating: test/Forest_CDN_3.jpg   
  inflating: test/Forest_CDN_4.jpg   
  inflating: test/Forest_CDN_5.jpg   
  inflating: test/Forest_CDN_6.jpg   
  inflating: test/Forest_CDN_7.jpg   
  inflating: test/Forest_CDN_8.jpg   
  inflating: test/Forest_CDN_9.jpg   
  inflating: test/Forest_CEB.jpg     
  inflating: test/Forest_CEB_1.jpg   
  inflating: test/Forest_CEB_10.jpg  
  inflating: test/Forest_CEB_11.jpg  
  inflating: test/Forest_CEB_12.jpg  
  inflating: test/Forest_CEB_13.jpg  
  inflating: test/Forest_CEB_14.jpg  
  inflating: test/Forest_CEB_15.jpg  
  inflating: test/Forest_CEB_16.jpg  
  inflating: test/Forest_CEB_17.jpg  
  inflating: test/Forest_CEB_18.jpg  
  inflating: test/Forest_CEB_19.jpg  
  inflating: test/Forest_CEB_2.jpg   
  inflating: test/Forest_CEB_3.jpg   
  inflating:

# 4. Predicción

Definir función que haga las predicciones con el modelo sobre el set de test.

In [14]:
ls

model_2048_512_1024/  model.zip  sample_data/  test/  test.zip


In [23]:
img_width = 224
img_height = 224

In [19]:
def make_predictions(model,
                     path_test ='test/',
                     name_file_submission = 'submission.csv'):

  images_test = os.listdir(path_test)
  samples_to_predict = []

  for img in images_test:
    path_img = os.path.join(path_test, img)
    img = image.load_img(path_img,
                        target_size=(img_width, img_height))
    x = image.img_to_array(img)
    samples_to_predict.append(x)

  samples_to_predict  = np.array(samples_to_predict)
  predictions = model.predict(samples_to_predict)
  classes = np.argmax(predictions, axis = 1)

  data = {'id.jpg': [img for img in images_test], 'label': classes}
  data = pd.DataFrame(data)
  data.to_csv(name_file_submission, index = False)

  return data



Hacer la predicción.


In [24]:
make_predictions(model, name_file_submission = 'resultados.csv' )

,id.jpg,label
0,Forest_ODN_15.jpg,14
1,Marine_Wat_15.jpg,18
2,Forest_DDN_14.jpg,10
3,Open_Shrub_0.jpg,21
4,Swamp_add__9.jpg,23
...,...,...
366,Forest_DEN_1.jpg,7
367,Marine_Wat_14.jpg,18
368,Marine_Wat_0.jpg,18
369,Snow_100_12.jpg,22


# 5. Descarga de resultados

In [25]:
from google.colab import files

files.download("resultados.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>